<a href="https://colab.research.google.com/github/audrie-z/Collecting-data/blob/main/htrc-yesminister.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Extracted Features to topic model with HTRC FeatureReader and Gensim
This notebook will introduce [HTRC Extracted Features](https://wiki.htrc.illinois.edu/x/kYC2B) files, the [HTRC FeatureReader](https://github.com/htrc/htrc-feature-reader) Python library, and demo how EF files can be used to topic model a workset (collection) of volumes using [Gensim](https://radimrehurek.com/gensim/). This can be helpful both as exploratory data analysis, since anyone can download EF files whenever they'd like, as well as more robust analysis, as Gensim is a powerful topic modeling/machine learning library. This notebook will introduce a few new functions to reformat EF files to a format that can be fed directly into a topic modeling library, such as Gensim or MALLET, as well as some very entry-level visualization of our generated topics using [pyLDAvis](https://github.com/bmabey/pyLDAvis).

Google Colaboratory (Colab) comes with a number of Python libraries built-in, but there are a few it doesn't have. These we will install directly on our Google server that runs this notebook, using `pip` a command line Python package manager. 

**NOTE: There will be a warning after running this cell that produces a "restart runtime" button. Do not worry about the warning, however, you will need to click on the button for the rest of the code in the notebook to work.**

In [1]:
!pip install htrc-feature-reader
!pip install pyLDAvis

     |████████████████████████████████| 58 kB 4.1 MB/s 
     |████████████████████████████████| 1.5 MB 15.7 MB/s 
     |████████████████████████████████| 155 kB 10.5 MB/s 
  Created wheel for htrc-feature-reader: filename=htrc_feature_reader-2.0.7-py3-none-any.whl size=40428 sha256=094a689f653faa5fe94a3fc05909f07bc2d155361cb6ac67d414b47e3747d9a7
  Stored in directory: /root/.cache/pip/wheels/22/18/9b/a096fa215c4f3703ed92bc7e0d50774c744685295415bc0ac9
Successfully built htrc-feature-reader
     |████████████████████████████████| 1.7 MB 8.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 11.3 MB 50.1 MB/s 
     |████████████████████████████████| 15.7 MB 26.3 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=753220f92845b644e251140daad4a38c4349e0ae76addb511d6b010040d37874
  S

First, as almost always with a Jupyter notebook, we'll import the libraries that we'll be using. We need to do this in order to tell Python which libraries have the commands/methods we'll be using:

In [2]:
import pandas as pd
from htrc_features import FeatureReader, Volume
from tqdm.notebook import trange, tqdm
import os
import shutil
import nltk
import gensim

# Because some of our libraries will throw many, many warnings for future changes to their code:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

We've imported:
- [Pandas](https://pandas.pydata.org/) a common library for working with large data, especially text data, while doing data science-y things.
- the HTRC FeatureReader, a specialty Python library for working with EF files.
- [TQDM](https://pypi.org/project/tqdm/) a library that allows for quick and easy progress bars for longer computational tasks.
- `NLTK`, the [Natural Language Toolkit](https://www.nltk.org/)
- Two built-in Python libraries, `shutil` and `os` which help us deal with files and file systems

You can think of importing libraries as basically getting our tools into our workspace, because we know that for some tasks we'll need a screwdriver and for others we may need some duct tape or a drill.


## Working with Volumes

We'll first use the FeatureReader to quickly take a look at EF data. The HTRC FeatureReader is a Python library that was written to make working with the EF files, especially for exploratory analysis, easier. The FeatureReader library aggregates information at a collection and volume level, and also has functions/methods at both levels. For this notebook, we'll be using the EF data at the volume level, though iterating through multiple volumes.

When using the FeatureReader, the first step is usually to create a `Volume` object out of one EF file. We're going to do this by giving the method `Volume()` a HathiTrust ID (HTID), but, if we had a lot of volumes we wanted to look at EF data for, we could also give it a file path to an EF file we already downloaded to our own computer:

In [3]:
test_vol = Volume('mdp.39015018983885')
print(f"{test_vol.title}, by {test_vol.author}, {test_vol.pub_date}, {test_vol.id}")

The complete Yes, Minister : the diaries of a cabinet minister /, by ['Lynn, Jonathan.', 'Jay, Antony, 1930-2016.'], 1989, mdp.39015018983885


Above we see that we created a FeatureReader `Volume` and we use a few built-in methods for a `Volume` to return metadata about the book--author, title, pub date, and HTID. This metadata is bibliographic, and was assigned by a librarian or cataloger at the institution that submitted the item to HathiTrust. This is handy because it can be used as ground truth for machine learning tasks as well as used during the text analysis pipe line, such as for de-duplicating volumes. There are more metadata fields available than above, and you can see all the options by typing `test_vol.` below then hitting `Tab`, which will trigger a pop-up menu listing the possible methods available. See some samples and explore other fields below:

In [4]:
# returns HT record number (which can help with de-duplicating)
print(test_vol.oclc)

# returns publisher information for the volume
print(test_vol.publisher)
 
# returns a code for the institution that submitted the item to HathiTrust
print(test_vol.source_institution)

# EXPLORE OTHER AVAILABLE METHODS/FIELDS
print(test_vol.date_created)
print(test_vol.genre)

35255310
BBC Books
MIU
20200209
['document (computer)']


Some of the methods that pop-up in the above cell are more complex than just returning metadata fields in the file. Some of the methods will instead return new Pandas DataFrames (basically big, interactive tables of data that we'll be using a lot today) with information. For example, we can use `test_vol.begin_line_chars()` to return a DataFrame of the characters that start each line on a given page. Explore some of the other options in the cell below by commenting or un-commenting the code lines (by adding/removing a `#` in front of the line):

In [6]:
# returns a DataFrame (DF) with begin-line characters for each page
test_vol.begin_line_chars()

# returns a DF with line counts for each page
# test_vol.line_counts()

# returns a DF with empty line counts for each page
# test_vol.empty_line_counts()

# EXPLORE OTHER METHODS!




AttributeError: ignored

                         count
page section place char       
2    body    begin -         1
                   /         1
                   :         1
                   ^         1
                   k         1
...                        ...
542  body    begin D         1
                   T         1
                   U         1
545  body    begin B         1
                   I         1

[10177 rows x 1 columns]

Beyond just generating these DataFrames, we can also plot many of them very quickly, which helps us better visualize a large amount of data. For instance, if we wondered about the textual structure of our volume, we may use `.tokens_per_page()` to visualize how many words are on each page of our volume, and see if there is anything interesting:

In [7]:
tkns_df = test_vol.tokens_per_page()
tkns_df.plot(figsize=(22,10))

# if you want to compare to another volume, here is code for looking at Mrs. Dalloway by Virginia Woolf:
# md_vol = Volume('mdp.39015002299421')
# md_tkns = md_vol.tokens_per_page()
# md_tkns.plot(figsize=(22,10))

ImportError: ignored

Anything interesting here?

Similarly, we can use the FeatureReader for more complex exploratory analysis. Maybe we suspect that parts of our volume contained poetry, we may wish to see if there are any perceivable increases or decrease of capitalized begin line characters in our volume. We could check this with the FeatureReader and a handful of lines of code:

In [8]:
# Create a new Volume from one anthology of Harper's Weekly, a publication known to publish both prose and poetry:
blc_test_vol = Volume('coo.31924054824473')

# create a DF with begin line characters only:
blc_df = blc_test_vol.begin_line_chars()
# blc_df.head(10)

# print the shape of our blc DF
print(f"blc_df has {blc_df.shape[0]} rows and {blc_df.shape[1]} columns")

# make a new DF with only capitalized alphabetic begin line characters, by page:
caps_blc_df = blc_df.loc[(slice(None), slice(None), slice(None), ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z'],)]

# plot the above df!
caps_blc_df.plot(figsize=(20,10))

blc_df has 27456 rows and 1 columns


ImportError: ignored

This plot is showing us the counts of capitalized begin line characters per page, and shows spikes around page 240-300, and then later in the volume. This shows where we are likely to have poetry in this volume, and happens to line up with more rigorous poetry-identifying work we've done in [another EF use case](https://wiki.htrc.illinois.edu/x/IQKGAQ).

## Topic Modeling with Token Lists

At this point, you're probably wondering when we'll get to topic modeling using the tokens (words) themselves. Let's not delay any longer and dive in. 

Before we topic model, let's look a bit at how we can see and interact with tokens. The most flexible way to do this is to create a token list DataFrame (DF) using the method `vol.token_list()`. This method can be given extra options to get different versions of the data back, and these are called "parameters." Let's take a look at the possible parameters first (maybe learning a handy trick), and then explore a token list. To see the possible parameters for a given function or method, you can type its name as if you were going to run it, but with a `?` instead of parentheses `()`:

In [9]:
# test_vol.tokenlist?

tv_pf = test_vol.page_features()
tv_pf.head(25) 

AttributeError: ignored

                               version calculatedLanguage
page                                                     
1     7215ee9c7d9dc229d2921a40e899ec5f               None
2     b779d7850ea63fb5dea523c5eb055686                 cy
3     d41d8cd98f00b204e9800998ecf8427e               None
4     7215ee9c7d9dc229d2921a40e899ec5f               None
5     6ddc3d6c540bab5c48fc8f75c6662851                 en
6     7215ee9c7d9dc229d2921a40e899ec5f               None
7     a6a4c61f7d0e583abd930ecb52042455                 en
8     008a59b28727335f9138b53b61662f5f                 en
9     ebc498030501231540a4cb3a97a672a9               None
10    e6879d6301880737aeb72efdc8ddf987               None
11    a0c85f28d90bea718b2330c1f66dd871                 en
12    40d3cbe361fc07680d247c96917d5380                 en
13    00e9adc2194e6622eda05a4b24b130f2                 en
14    d8d2f9519142ccc6f635019cbb78f8e7                 en
15    fdcbf23d70a15cbde123e16cd8d7d70e                 en
16    691cd2c8

The above code will not only show us the parameters, but will return the entire Docstring help text for the function, which should not only tell you possible parameters, but also their default values, the data types the function and each parameter takes, the output of the function and even examples of how to use it and what it will return. One thing to keep in mind, though, is that this info is submitted by the author of the library, so how thorough or helpful this help text is can vary wildly between libraries!

Ok, with a new trick under our belts, let's create our first token list to see what info is available. While we do this, we'll also assign the output of `.tokenlist()` to a variable and use `.head()` to look at the first 5 rows of data, since we know the output of `.tokenlist()` is a DataFrame:

In [10]:
test_df = test_vol.tokenlist()
test_df.head()

AttributeError: ignored

                        count
page section token pos       
2    body    "     UNK      1
             *:\.  UNK      1
             -     UNK      1
             /^    UNK      1
             :     UNK      1

Since we looked at an EF file, this data should look familiar, but the structure has been flattened a bit to fit into a DataFrame, which can make it easier to read to our human (hopefully!) eyes. Another interesting thing to note is that the DataFrame returned by `.tokenlist()` has a multi-index. This means that more columns than just a hidden numerical index are treated like an index--that is, you can easily search those levels of data for certain values.This is why we could use `.loc` to quickly find certain letters in our capitalized begin line character example above. This is a powerful way to find certain tokens or parts-of-speech or to do so on specific pages or sections of a volume, though we won't demo that here.

Instead, let's turn directly to using Extracted Features files to topic model our data. Right now, we can get the tokens for each page in a given HathiTrust volume in a highly structured format, a token list. Most topic modeling tools simply want the tokens for each volume/document/page in bag-of-words format. You can think of the bag-of-words format exactly as it sounds:

![](https://drive.google.com/uc?export=view&id=1LHus0vtaiKxKIBIyimtSJOjvqaH4O6Oi)

We take each word from a passage of text, drop it in a "bag" where it loses its original order but still represents the explicit pieces of the original text. Topic modeling is based on co-occurrence between words and word occurrence at the document and topic level, which is why the original semantic structure of the text is irrelevant.

In order to convert EF files to bag-of-words text, we need to write a bit of custom code. We'll define a new function that will parse EF files, grab the tokens for each page and volume and add them to a large DataFrame that will have bag-of-words tokens as one of its columns, along with HTID and page number and a few others. We call this function `ef_vol_to_bow_df` and it takes a FeatureReader `Volume` as an input as well as an optional parameter to save the DataFrame to a TSV. Since this function takes one volume as an input, we'll need to wrap it in some iteration code in order to reformat multiple volumes. Here is the function:

In [11]:
def ef_vol_to_bow_df(volume, save_to_tsv=False):
    from htrc_features import FeatureReader, Volume
    import pandas as pd
    from tqdm.notebook import trange, tqdm
    import os
    notebook_path = os.getcwd()
    htid = volume.id
    # creating an empty Dataframe to which we'll add clean tokens, instead of a text file
    vol_df = pd.DataFrame(columns=['htid', 'page_number', 'page_tokens'])
    outfile_name = htid+'.tsv' # saving our volume-level token DataFrame to a TSV
    for page in tqdm(volume.pages(), total=volume.page_count):
        page_num = str(page).split(' ')[1]
        page_df = page.tokenlist(section='body', case=False, pos=False)
        tkn_list = []
        
        for i, r in page_df.iterrows():
            tkn = i[2]
            clean_tkn = tkn.strip()
            count = r[0]
            tkns = ([f'{clean_tkn}'] * count)
            clean_tkn_list = [word for word in tkns if word.isalpha()]
            clean_tkn_list = [word for word in clean_tkn_list if word not in en_stop]
            tkn_list.extend(clean_tkn_list)
        '''
        Instead of writing to text files, we are adding the page-level clean tokens to our
        DataFrame, with one each page of tokens constituting one row in the DataFrame.
        '''
        vol_df = vol_df.append({'htid': htid, 'page_number':  page_num, 'page_tokens':tkn_list}, ignore_index=True)
    '''
    Lastly, we save our volume-level DataFrame as a tab-separated file, and return the volume 
    DataFrame so that we can better aggregate each volume's tokens into a single DataFrame (you'll 
    see this code in the wrapper we write to iterate through multiple volumes)
    '''
    if save_to_tsv==True:
        vol_df.to_csv(outfile_name, sep='\t', index=False)
        print(f'Saved {volume.title} to TSV named {outfile_name}')
    
    print(f'Reformatted "{volume.title}" ({htid}) to bag-of-words')
    return vol_df

Before we run this function to test it out, we will download a standard set of stop words--the words we want to remove from the text before analysis--and decide if we want to add any additional words to this list. For this workshop, we've built removing stop words into the function that readies EF files for topic modeling. We'll use the standard English stop words from the Natural Language Toolkit (`nltk`) to start with. We have cheated (by running this already) to find that we should make some additions, which we'll do using `.add()` the syntax for adding items to a `Set` (a `Set` is basically a list, but where every item in it is unique and with some extra properties):

In [12]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

en_stop.add("'")
en_stop.add('"')
en_stop.add(' ')
en_stop.add('would')
en_stop.add('could')
en_stop.add('should')
en_stop.add('said')
en_stop.add('also')

# If we wanted to be a bit more clever, we could use a simple loop to add these words to our stop list with less typing:

# stop_words_to_add = ["'",'"',' ','would','could','should','also', 'said']

# for word in stop_words_to_add:
#     en_stop.add(word)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


With a list of stop words defined, we can write a few lines of code to iteratively deploy it and test the results. We'll first use three HTIDs for test volumes (the first three books of George RR Martin's *A Song of Ice and Fire* series) and then create a `for` loop to run our `ef_vol_to_bow_pages` function over all three volumes:

In [17]:
# YOU COULD SUBSTITUTE ANY VOLUMES YOU'D LIKE HERE, JUST REPLACE THE BELOW HTIDs!
volume_list = ['mdp.39015050507618','mdp.39015046463629','mdp.39015018983885']

workset_page_df = pd.DataFrame(columns=['htid','page_number','page_tokens'])

for book in tqdm(volume_list):
    fr_vol = Volume(book)
    book_df = ef_vol_to_bow_df(fr_vol, save_to_tsv=True)
    workset_page_df = workset_page_df.append(book_df)
    
print(f"Reformatted {len(volume_list)} volumes to bag-of-words pages.")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/876 [00:00<?, ?it/s]

ImportError: ignored

Things look to have run successfully (hopefully!). Let's check the dimensions (rows, columns) and look at the output DataFrame to verify:

In [18]:
print(workset_page_df.shape)
workset_page_df.head(10)

(0, 3)


AttributeError: ignored

Empty DataFrame
Columns: [htid, page_number, page_tokens]
Index: []

The dimensions look right, as does the data in `workset_page_df`. 

We can now shift to grabbing the bag-of-words text in the `page_tokens` column and readying it for Gensim. We're using a straightforward, no frills version of topic modeling where we aren't tracking the documents in our collection, and are instead just topic modeling the words. For this, Gensim wants one big list of the tokens, and we'll generate this using another `for` loop:

In [19]:
bow_text_lists = []
bow_text = []

for i,r in workset_page_df.iterrows():
    words = r.page_tokens
    bow_text_lists.append(words)
    for word in words:
        bow_text.append(word)

print(f"Added {len(bow_text)} words to list.")

Added 0 words to list.


Gensim takes "dictionary" and "corpus" objects to train an LDA model. For Gensim, a "dictionary" is a list of the unique tokens in your corpus, assigned to a numerical ID, and a "corpus" is your complete bag-of-words data--the list of words we just created, `bow_text`--but represented by the numerical ID from the "dictionary" and its count. This is a bit confusing, so let's generate both and take a look at them directly. Here is the code to create the dictionary and corpus, pulled from Gensim documentation:

In [20]:
# importing libraries from Gensim to create corpus and dictionary
from gensim import corpora

# creating our dictionary and corpus
dictionary = corpora.Dictionary(bow_text_lists)
corpus = [dictionary.doc2bow(text) for text in bow_text_lists]

# importing pickle, a library which lets us save our corpus and dictionary for later use
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

Here is a look at the first 15 items in our Gensim `dictionary`:

In [21]:
n = 0

for k,v in dictionary.items():
    if n <= 15:
        print(f"{k} : {v}")
        n += 1

And here is one page of bag-of-words data (represented numerically as `(word ID, count)` pairs or tuples) in our Gensim `corpus`:

In [25]:
# return item 5 in our corpus:
corpus[5]

IndexError: ignored

With these two inputs created, we can now train and run our topic model, saving the model along the way (again, pulling code from Gensim documentation--you don't have to know how to do this off the top of your head!):

In [26]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# THIS CODE WILL TAKE A FEW MINUTES OR SO TO RUN!
number_of_topics = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = number_of_topics, id2word=dictionary, passes=50, random_state=42)
ldamodel.save('model10.gensim')

ValueError: ignored

Three major inputs are adjustable: the number of topics we want to generate, the number of passes we want to do when training our model, and the random state or seed. The number of topics is custom for each set of volumes/documents and is often something that you should test out. Do your results make more or less sense with more or fewer topics? The number of passes will effect the "accuracy" of your topic models. The algorithm is training the topics as many times as you input passes, and then taking an aggregated average of the results. Since there is a random element to topic modeling, results are different for every pass. The more passes we do, the more we can reduce the randomness of the results. Here, we're doing 50 passes, since our data is not too big, but you may find more or less is suitable based on testing out results. Another way to minimize the randomness is to set the seed, or in Gensim the `random_state`. By setting the "seed" or the `random_state` we are saying to eliminate the randomness and always go with the number option we indicate. You can play with this number as well and see how the results change. We have started with 42 because it is the answer to eveything.

Ok, back to results!

Nothing is explicitly returned from the code above, as the model is created and saved, but we can see the topics that were generated using `show_topics()` which also lets us decide how many of the top words for each topic we want to see:

In [27]:
topics = ldamodel.show_topics(num_words=10, formatted=False)
for topic in topics:
    print(f"{topic} + '\n'")

# doc_topics = ldamodel.get_doc_topics

NameError: ignored


We can also get all the documents for which a term has an association in our model using Gensim's `get_term_topics` method, which takes a term and a minimum probability and returns each topic for which that term meets the minimum probability of appearing in. We can see all topics for a given term simply by passing `0` to the parameter `minimum_probability`:



In [28]:
ldamodel.get_term_topics('brave', minimum_probability=0)


NameError: ignored

While it's very useful to see our topics, the top N terms associated with them, and be able to check topic association for a given term, we may also want to see how each of our documents associates with our topics. We can use Gensim's `get_document_topics` method for this (and remember that a 'document' for us is a page of a HathiTrust volume):

In [29]:
all_topics = ldamodel.get_document_topics(corpus, per_word_topics=True)
n = 0

for doc_topics, word_topics, phi_values in all_topics[20:25]:
    print(f"Document {n}\n")
    print('Document topics (what topics this document is associated with):')
    print(f"{doc_topics}\n")
    print('Word topics (what topics words in this document are associated with):')
    print(f"{word_topics}\n")
    print('Phi values (probability of a word in this document being associated with a given topic):')
    print(f"{phi_values}\n")
    print('-------------- \n')
    n += 1

NameError: ignored

We can also Write out the topics and their top n words (here, n=25) to a file with the following code (**Note: this doesn't work in Google Colab**):

In [30]:
# DOESN'T WORK FOR COLAB, SADLY!
topics = ldamodel.show_topics(num_topics=5, num_words=25)

with open('asoiaf_topics_5.tsv', 'a') as f:
    for topic in topics:
        topic_num = topic[0]
        topic_words = topic[1]
        f.write(f"{topic_num} + '\t' + {topic_words} + '\n'")

NameError: ignored

Now, with our topic modeling results, the fun begins--interpreting what we see. For this set of books, *A Game of Thrones*, *A Clash of Kings* and *A Storm of Swords*, the results *seem* to make sense, as we see the clustering of characters that interact with each other heavily, as well as thematic elements for those characters and their narrative arcs.

We can use another library called `pyLDAvis` to better visualize our topics, and see if they still make sense:

In [31]:
# load the dictionary, corpus, and LDA model we created earlier:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))

# If you generate a new model and change the number of topics, you may need to change the file name for the model (here, model5.gensim)
lda = gensim.models.ldamodel.LdaModel.load('model10.gensim')

# import pyLDAvis and ready it for use in a notebook:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed pyLDAvis the pieces generated from Gensim and create the visualization:
lda_display = gensimvis.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

FileNotFoundError: ignored

That's it! You've topic modeled 3 books, tens of thousands of words, using Gensim and HTRC Extracted Features. However, since topic modeling has a random seed, you should continue to explore by tweaking the inputs: stop words, number of topics, passes, and the random seed itself. We could also topic model while retaining the structure of the original pages/documents, or chunking our text into smaller pieces and reviewing how the results might change. Since topic modeling is really great for exploring data, we shouldn't be shy when it comes to experimenting with inputs and data. You'll never know how good results can be until you try different methods of generating them.

**Questions about this notebook?** Contact HTRC at htrc-help@hathitrust.org

CC-BY-NC 4.0 (2021)